# Persona-Hub による SFT データ合成

Susumu Ota  2025-02-02

## 背景: 合成データの重要性

TODO: 以前の講演資料から抜粋

## 背景: 合成データ作成の難しさ

TODO: 以前の講演資料から抜粋

## 目的

TODO: 以前の講演資料から抜粋

## モジュールのインストール

`litellm` と `datasets` をインストールしてください。ローカルのGPUを使って推論する場合は `vllm` もインストールしてください。API だけを使う場合はインストール不要です。

In [1]:
# %pip install litellm
# %pip install datasets

# %pip install vllm

## モジュールのインポート

In [2]:
from abc import ABC, abstractmethod
import json
from logging import DEBUG, INFO, StreamHandler, getLogger  # noqa: F401
import pprint
import random
import re


from datasets import load_dataset
from litellm import batch_completion

try:
    from vllm import LLM, SamplingParams  # type: ignore
except ImportError:
    print("No vllm module found. You can only use the LiteLLM.")

/Users/ota/Documents/python/synthetic-data/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ota/Documents/python/synthetic-data/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


No vllm module found. You can only use the LiteLLM.


## ログの設定

In [3]:
logging_level = DEBUG
# logging_level = INFO
logger = getLogger(__name__)
logger.setLevel(logging_level)
handler = StreamHandler()
handler.setLevel(logging_level)
logger.addHandler(handler)

pp = pprint.PrettyPrinter(width=119)

## 言語モデルの設定

OpenAI 等の API とローカル GPU を使った vLLM で統一してコードを書くために、`LanguageModel` という抽象クラスを作り、サブクラスで個別の処理を実装します。

- `__init__`: 言語モデルの初期化。推論用のパラメータ(`temperature`等)も設定。
- `__call__`: 推論の実行。入力は OpenAI messages 形式 (e.g. `[{"role": "user", "content": "Hello!"}]`) のリスト。出力は文字列のリスト。

In [4]:
class LanguageModel(ABC):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None):
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.seed = seed
        logger.debug(f"model: {model}, temperature: {temperature}, max_tokens: {max_tokens}, seed: {seed}")

    @abstractmethod
    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        pass


class LiteLLMModel(LanguageModel):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None):
        super().__init__(model, temperature, max_tokens, seed)

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        contents = [
            response.choices[0].message.content or ""
            for response in batch_completion(
                model=self.model,
                messages=messages_batch,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                seed=self.seed,
            )
        ]
        assert len(contents) == len(messages_batch)
        return contents


class VLLMModel(LanguageModel):
    def __init__(self, model: str, temperature=1.0, max_tokens=16, seed=None, dtype="auto", stop=None):
        super().__init__(model, temperature, max_tokens, seed)
        self.stop = stop
        self.vllm = LLM(model, dtype=dtype)
        self.tokenizer = self.vllm.get_tokenizer()

    def __call__(self, messages_batch: list[list[dict[str, str]]]) -> list[str]:
        sampling_params = SamplingParams(
            temperature=self.temperature, max_tokens=self.max_tokens, seed=self.seed, stop=self.stop
        )
        prompts = [
            self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            for messages in messages_batch
        ]
        outputs = self.vllm.generate(prompts, sampling_params=sampling_params, use_tqdm=False)
        contents = [o.outputs[0].text for o in outputs]
        assert len(contents) == len(messages_batch)
        return contents

## 言語モデルの初期化

以下のどれかのコメントを外して言語モデルを初期化します。API の場合は環境変数やシークレットマネージャーで API キーを設定してください。環境変数名やモデル名は、[LiteLLM の Providers ページ](https://docs.litellm.ai/docs/providers)を参照してください。

一般に API の推論速度は遅いので、予備実験を API で行い、本実験は vLLM で処理するという使い方が現実的かもしれません。

API で大量のデータを生成する場合は、バッチ処理のオプションが用意されていればそれを利用することと、利用制限(rate limit等)の範囲内でリクエストの並列化を検討してください。LiteLLM は内部でマルチスレッドでリクエストを並列で送るので、バッチサイズ大きめで使うと rate limit を超える可能性があります。

In [5]:
llm = LiteLLMModel("gpt-4o-mini", temperature=0.7, max_tokens=512, seed=0)  # OPENAI_API_KEY
# llm = LiteLLMModel("nvidia_nim/nvidia/llama-3.1-nemotron-70b-instruct", temperature=0.7, max_tokens=512, seed=0)  # NVIDIA_NIM_API_KEY
# llm = LiteLLMModel("nvidia_nim/nvidia/nemotron-4-340b-instruct", temperature=0.7, max_tokens=512, seed=0)  # NVIDIA_NIM_API_KEY
# llm = LiteLLMModel("deepinfra/nvidia/Llama-3.1-Nemotron-70B-Instruct", temperature=0.7, max_tokens=512, seed=0)  # DEEPINFRA_API_KEY
# llm = VLLMModel("hpprc/gemma-2-2b-jpn-it", temperature=0.7, max_tokens=512, seed=0, stop=["<end_of_turn>"], dtype="half")  # ローカル T4

llm

model: gpt-4o-mini, temperature: 0.7, max_tokens: 512, seed: 0


## 言語モデルの動作確認

ここまで設定できれば以下の推論が動作するはずです。もしエラーが出る場合は、言語モデルの初期化の設定を見直してください。

In [6]:
llm([[{"role": "user", "content": "Hello?"}]])

['Hello! How can I assist you today?']

## ペルソナとは

https://huggingface.co/datasets/argilla/FinePersonas-v0.1 より引用

> - 個人の特徴、背景、目標を詳細に記述したもので、多様なアイデンティティと経験を反映するようにデザインされている
> - ペルソナの例
>   - A network engineer with a focus on routing protocols and preparing for Cisco certification exams, particularly CCNA.
>   - ルーティング・プロトコルに興味があり、シスコの認定試験(特に CCNA)の準備をしているネットワーク・エンジニア
> - 生成するコンテンツに、特定の専門知識・キャリアパス・個人的な興味を導入し、より繊細でターゲットを絞ったコンテンツが生成可能

## Persona-Hub とは

<img src="https://github.com/tencent-ailab/persona-hub/blob/main/assets/persona_overview.png?raw=true" width="800px">

- Tencent AI Lab が提案したペルソナ駆動型データ合成手法で、大規模言語モデル内の様々な視点を活用して多様な合成データを作成
- 背景: 既存の合成データ⽣成⼿法(インスタンス駆動・キーポイント駆動)では合成データの多様さをスケールアップすることが困難
- 目的: ⼤規模なペルソナコレクションを作成し、それを使ってスケーラブルな合成データを⽣成する(ペルソナ駆動)
- 方法
  - Web データからペルソナを抽出
    - Text-to-Persona
    - Persona-to-Persona
  - ペルソナを使って合成データを生成
    - Create **a math problem** with **a moving company driver**
    - Create **a math problem** with **a chemical kinetics researcher**
    - Create **a math problem** with **a musician interested in audio processing**
- テクニカルレポート: https://arxiv.org/abs/2406.20094

## ペルソナの抽出

ここで実際に FineWeb-Edu データセットからペルソナを抽出します。まずデータセットを読み込みます。

In [7]:
fineweb_edu = load_dataset("HuggingFaceFW/fineweb-edu", name="CC-MAIN-2024-51", split="train", streaming=True)
fineweb_edu

IterableDataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count', 'score', 'int_score'],
    num_shards: 50
})

今回は実験のため、FineWeb-Edu データセットの先頭の10件を取り出します。

In [8]:
web_pages = [d["text"] for d in fineweb_edu.take(10)]
web_pages

["This project is solving the Asteroid Watchers challenge. Description\nAROs are essentially created by combining a telescope with a smartphone. If the telescope has drive motors can be controlled via a standard protocol, it is controlled by the SkyWatch app on the smartphone. Telescopes with no drive motors of their own need to have these and a controlling Arduino added. We plan to create or adopt a small reference / demo model of this an a open source hardware and software sub-project, where the smartphone controls the Arduino to operate this telescope's stepper motors.\nAll AROs, their location and capabilities are registered in the Global ARO Co-Ordinator (GAROC) database.\nBasic Variant: This version allows the ARO owner to build the hardware and get it up and running and connected into the SkyWatch network. The focus here is to get various simple, low-power home-built and commonly sourced telescopes into the network, for maximum global participation. The system caters for telesco

上記のように教育分野の比較的品質の高い Web ページを集めたデータセットで、科学技術分野のペルソナを抽出できると期待されます。

### Text-to-Persona

テキストからペルソナを抽出するには以下のようなプロンプトを実行します。日本語で出力するように指示していますが、品質が低い場合は `Note:` 以下を削除して英語で出力してください。

In [9]:
def text_to_persona(llm: LanguageModel, text: str) -> str:
    SYSTEM_PROMPT = """\
You are an expert in analyzing the text content and assigning finding the general type of persona that could be associated with such a way of expressing. Please use one or two sentences for the definition, but try to make it as fine-grained if input texts involve many detailed elements. The persona definition must go straight to the point, be assertive. The following are starts of persona definitions:
A machine learning researcher...
A pedriatric nurse whose...
An urban planner focused on...
"""

    USER_PROMPT = """\
What is the likely profession, interest, or role of the person who would write or be interested in this text?

## Text
{text}

Note:
1. Your response should always start with "ペルソナ:".
2. 日本語で回答してください。
"""
    persona = llm([[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT.format(text=text)},
    ]])[0]

    return re.sub(r"^ペルソナ:", "", persona).strip()

FineWeb-Edu データからペルソナを抽出します。

In [10]:
persona = text_to_persona(llm, web_pages[2])
pp.pprint({"persona": persona, "text": web_pages[2]})

{'persona': 'IoTおよびAI技術に精通したエンジニアで、データ分析やシステムの最適化に関心を持つ、技術革新を推進する専門家。',
 'text': 'The internet of things, a system of interrelated computing devices and machines that can transfer data over '
         'a network without human interaction, has been used to enable new features, better functionality, and '
         'real-time status monitoring for consumers. Combining this with ever-developing AI advancements is allowing '
         'organizations to predict changes and optimize their devices. AIoT allows an algorithm to improve '
         'communication and apply predictive capabilities to give companies advantages over their competition.\n'
         'IoT devices share the sensor data they collect by connecting to an IoT gateway or other edge device where '
         'data is either sent to the cloud to be analyzed or analyzed locally. Sometimes, these devices communicate '
         'with other related devices and act on the information they get from one another. The devices do most of the

### Persona-to-Persona

Web ページに出現する頻度が低いペルソナを見つけるために、得られたペルソナから関連するペルソナを生成します。

In [11]:
def persona_to_personas(llm: LanguageModel, persona: str) -> list[str]:
    SYSTEM_PROMPT = """\
You are an AI assistant expert in finding relationships between people. Answer directly with the the new related persona definition, don't enumerate them.
"""

    USER_PROMPT = """\
Who is in close relationship with the given persona? Write just 3, each one in a different line:
{persona}

Note:
1. Your response should always start with "ペルソナ:".
2. 日本語で回答してください。
"""

    persona = llm([[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT.format(persona=persona)},
    ]])[0]

    return [p.strip() for p in re.sub(r"^ペルソナ:", "", persona).strip().split("\n")]

In [12]:
new_personas = persona_to_personas(llm, persona)
pp.pprint({"org_persona": persona, "new_personas": new_personas})

{'new_personas': ['技術革新を推進するスタートアップのCEO', 'データサイエンティストとして働く同僚', 'IoT関連のプロジェクトマネージャー'],
 'org_persona': 'IoTおよびAI技術に精通したエンジニアで、データ分析やシステムの最適化に関心を持つ、技術革新を推進する専門家。'}


## ペルソナを使って合成データを生成

ここでは、ペルソナを使ってユーザが質問しそうなプロンプト(インストラクション)を生成します。

In [13]:
def generate_instruction(llm: LanguageModel, persona: str) -> dict[str, str]:
    SYSTEM_PROMPT = "You are an AI assistant expert at simulating user interactions."

    USER_PROMPT = """\
Generate a prompt the persona below might ask to an AI assistant:

{persona}

Note:
1. Your response should always start with "プロンプト:".
2. 簡潔に日本語で回答してください。
"""

    instruction = llm([[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT.format(persona=persona)},
    ]])[0]

    return re.sub(r"^プロンプト:", "", instruction).strip()

In [14]:
instruction = generate_instruction(llm, persona)
pp.pprint({"persona": persona, "instruction": instruction})

{'instruction': 'IoTデバイスから収集したデータを最適に分析するためのアルゴリズムやツールについて教えてください。',
 'persona': 'IoTおよびAI技術に精通したエンジニアで、データ分析やシステムの最適化に関心を持つ、技術革新を推進する専門家。'}


次に、ペルソナを使ってブログデータを生成します。

In [15]:
def generate_quora_post(llm: LanguageModel, persona: str) -> str:
    SYSTEM_PROMPT = "You are an AI assistant specialized in writing posts for social media."

    USER_PROMPT = """\
Write a Quora post in the language, style, and personality of the following persona:

{persona}

Note:
1. Your response should always start with "タイトル:".
2. 簡潔に日本語で回答してください。
"""

    post = llm([[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT.format(persona=persona)},
    ]])[0]

    return re.sub(r"^タイトル:", "", post).strip()

In [16]:
post = generate_quora_post(llm, persona)
pp.pprint({"persona": persona, "post": post})

{'persona': 'IoTおよびAI技術に精通したエンジニアで、データ分析やシステムの最適化に関心を持つ、技術革新を推進する専門家。',
 'post': 'IoTとAIがもたらすデータ分析の未来\n'
         '\n'
         'IoT（モノのインターネット）とAI（人工知能）の融合は、データ分析の世界に革命をもたらしています。これにより、リアルタイムでのデータ収集と解析が可能になり、企業は迅速な意思決定を行えるようになりました。\n'
         '\n'
         '例えば、スマートシティにおけるIoTデバイスは、交通状況やエネルギー消費を監視し、AIを使って最適なリソース配分を実現します。これにより、効率的な運用が可能となり、環境への負荷も軽減されるのです。\n'
         '\n'
         'さらに、製造業においても、IoTセンサーが機械の状態を常時監視し、AIが故障予測を行うことで、メンテナンスコストの削減と稼働率の向上が期待できます。\n'
         '\n'
         'このように、IoTとAIはデータ分析の可能性を広げ、さまざまな分野でのイノベーションを加速させています。今後もこのトレンドは続くでしょう。皆さんは、どのような分野でこの技術を活用していくべきだと考えますか？'}


最後に、ペルソナを使って数学の問題を生成します。

In [17]:
def generate_math_problem(llm: LanguageModel, persona: str) -> str:
    SYSTEM_PROMPT = "You are an AI assistant specialized in creating diverse but specific math problems. Just answer with your problem."

    USER_PROMPT = """\
Create a challenging math problem with the following persona:

{persona}

Note:
1. Your response should always start with "問題:".
2. 簡潔に日本語で回答してください。
"""

    problem = llm([[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT.format(persona=persona)},
    ]])[0]

    return re.sub(r"^問題:", "", problem).strip()

In [18]:
problem = generate_math_problem(llm, persona)
pp.pprint({"persona": persona, "problem": problem})

{'persona': 'IoTおよびAI技術に精通したエンジニアで、データ分析やシステムの最適化に関心を持つ、技術革新を推進する専門家。',
 'problem': 'あるIoTシステムが、センサーから1日に1000件のデータを収集します。このデータは、温度、湿度、圧力の3つの異なるパラメータで構成されています。システムは、これらのデータをリアルタイムで分析し、異常値を検知します。もし、異常値の検出率が85%で、誤検出率が10%である場合、1日のデータに対して異常と判断される件数は何件になりますか？また、実際に異常であったデータ件数が50件の場合、真陽性、偽陽性、真陰性、偽陰性の件数を求めてください。'}


## FinePersonas とは

<img src="https://cdn-uploads.huggingface.co/production/uploads/6435d564a4bd75c62cc03701/5wTHwgijTUKFI5B-N7gEg.png" width="800px">

- Argilla が2024年9月にリリースされた合成テキスト生成のための2100万人の詳細なペルソナのオープンデータセット
- 合成データの豊富さ・多様性・特異性を高めることが可能
- FineWeb-Edu(教育関連Webページのデータセット)から Persona-Hub 手法でペルソナを抽出
- ライセンス: Llama 3.1 Community License Agreement
- https://huggingface.co/datasets/argilla/FinePersonas-v0.1

## ペルソナデータセットの読み込み

ここからは実際のペルソナデータを読み込んで、数学の問題と解答を合成します。`argilla/FinePersonas-v0.1` データセットを利用します。

FinePersonas データセットは巨大なデータセットですので、今回は一部分だけをダウンロードして利用します。読み込む際に `streaming=True` を必ずつけてください。

In [19]:
dataset = load_dataset("argilla/FinePersonas-v0.1", split="train", cache_dir="cache", streaming=True)
dataset

IterableDataset({
    features: ['id', 'persona', 'labels'],
    num_shards: 12
})

今回は先頭の100件を使って生成します。

In [20]:
personas = [data["persona"] for data in dataset.take(100)]
pp.pprint(personas)

['A professional R programmer or researcher, likely a data analyst or statistician, familiar with the intricacies of '
 'the R language and its debugging tools.',
 'A mental health professional, likely a licensed therapist or psychologist, with expertise in anxiety disorders and '
 'cognitive-behavioral therapy, whose work involves diagnosing and treating patients with various types of phobias, '
 'including specific phobia, social phobia, and agoraphobia.',
 'A space roboticist or aerospace engineer at a research institution focused on robotic space exploration.',
 'A Hebrew language scholar or instructor with a focus on biblical Hebrew and Jewish studies.',
 'A pediatrician or healthcare professional focused on educating parents about early childhood health, or a '
 'parent-to-be who is interested in learning about vaccinations for their unborn or newborn child.',
 'A Montessori elementary school teacher or an education administrator responsible for curriculum development and '
 'cla

## FinePersonas によるマルチターン SFT データ合成

ここではマルチターンの対話データを生成します。具体的には、

- 質問1 (`Q1`): ユーザが質問をする (この部分を Persona-Hub 手法で生成)
- 解答1 (`A1`): アシスタントが解答する
- 質問2 (`Q2`): ユーザが追加質問をする
- 解答2 (`A2`): アシスタントが追加質問に解答する

という4つの対話データを生成します。このうち `Q1` の生成に Persona-Hub 手法を適用します。それ以降の対話データは、若干の工夫をしますが基本的に言語モデルの生成に任せます。

## Q1 の生成

まず、Persona-Hub 手法を使って `Q1` の生成を行います。

[元論文のコード](https://github.com/tencent-ailab/persona-hub/blob/main/code/prompt_templates.py)では、数学の問題を生成するプロンプトとして以下が紹介されています。

```python
math_template = '''Create a math problem related to the following persona:

{persona}

Note:

1. The math problem should be challenging and involve advanced mathematical skills and knowledge. Only top talents can solve it correctly.
2. You should make full use of the persona description to create the math problem to ensure that the math problem is unique and specific to the persona.
3. Your response should always start with "Math problem:". Your response should not include a solution to the created math problem.
4. Your created math problem should include no more than 2 sub-problems.
'''
```

これを参考に、プロンプトの一部を修正/パラメータ化して使います。修正点は以下の通りです。

- 一部の単語をパラメータ化
  - タスクの種類: `task` (e.g. "math", "reasoning", "coding")
  - トピックの種類: `topic` (e.g. "persona", "topic")
  - トピックの内容: `item` (e.g. "SF作家", "経営コンサルタント")
  - 対象者: `target` (e.g. "grade school student", "graduate student")
- 難易度を調整
  - 元のプロンプトはかなり難易度の高い問題を生成するような表現(`challenging`, `advanced`, `top talents`)となっていたため、難易度を下げるように表現を修正(`simple`, `basic`, `average`)
- 小問を生成しないよう修正
  - 小問を生成すると対話が複雑になりすぎるため、小問を生成しないように修正(`sub-problems`の行を削除)。代わりにマルチターンで追加質問をするようにする。
- 日本語を出力
  - 元のプロンプトは英語を出力するようになっているため、日本語を出力するように促すプロンプトを追加

上記の修正をしたプロンプトを OpenAI messages 形式で出力する関数を作成します。

利用する言語モデルで推奨されているシステムプロンプトがある場合は、それを指定します(コメントアウトされた `{"role": "system", "content": "..."}` の部分)。

In [21]:
def get_q1_prompt(task: str, topic: str, item: str, target: str) -> list[dict[str, str]]:
    Q1_PROMPT_TEMPLATE = """Create a {task} problem related to the following {topic}:

{item}

Note:

1. The {task} problem should be simple and involve basic {task} skills and knowledge. Any average {target} can solve it correctly.
2. You should make full use of the {topic} description to create the {task} problem to ensure that the {task} problem is unique and specific to the {topic}.
3. Your response should always start with "問題:". Your response should not include a solution to the created {task} problem.
4. 簡潔に日本語で回答してください。
"""
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": Q1_PROMPT_TEMPLATE.format(task=task, topic=topic, item=item, target=target)},
    ]

実行して動作を確認します。

In [22]:
q1_prompt = get_q1_prompt("math", "persona", random.choice(personas), "grade school student")
pp.pprint(q1_prompt)

[{'content': 'Create a math problem related to the following persona:\n'
             '\n'
             'A paleoclimatologist or oceanographer with a focus on mass extinctions and marine ecosystems, possibly '
             'affiliated with an academic or research institution.\n'
             '\n'
             'Note:\n'
             '\n'
             '1. The math problem should be simple and involve basic math skills and knowledge. Any average grade '
             'school student can solve it correctly.\n'
             '2. You should make full use of the persona description to create the math problem to ensure that the '
             'math problem is unique and specific to the persona.\n'
             '3. Your response should always start with "問題:". Your response should not include a solution to the '
             'created math problem.\n'
             '4. 簡潔に日本語で回答してください。\n',
  'role': 'user'}]


問題なさそうなので、言語モデルで推論して Q1 を生成します。

In [23]:
q1s = llm([get_q1_prompt("math", "persona", random.choice(personas), "grade school student")])
pp.pprint(q1s)

['問題: '
 '環境ジャーナリストであるあなたは、オーガニック農法で作物を育てる農家を取材しています。農家は、1エーカーの土地で年間に2000ポンドのオーガニック野菜を収穫します。また、彼は蜂の巣を5つ持っており、各蜂の巣から年間に100ポンドの蜂蜜を収穫します。農家がオーガニック野菜と蜂蜜を合わせて年間にどれだけの重さを収穫するかを計算してください。']


引数を変更して難易度の高い問題を生成します。

In [24]:
q1s = llm([get_q1_prompt("advanced math", "persona", random.choice(personas), "graduate student")])
pp.pprint(q1s)

['問題: '
 '中学校の英語教師であるあなたは、創造的な執筆と文学の授業を行っています。最近、学生たちに人気のある映画をテーマにした短編小説を書く課題を出しました。クラスには30人の生徒がいて、各生徒は平均して5ページの短編小説を書く予定です。\n'
 '\n'
 'もしその短編小説をすべて印刷するための用紙が必要だとしたら、1ページあたりの用紙の価格が0.05ドルである場合、すべての生徒の短編小説を印刷するのに必要な総コストはいくらになるでしょうか？']


問題さなさそうです。

次に、`問題:` など余分な部分をルールベース(正規表現)で削除します。言語モデルによっては、解答やヒントを書いてしまうものがありますので、もしそれらが出力されるのであればここで削除しておきます。

**Note: この関数が複雑になり過ぎる場合は、プロンプトの見直しや言語モデルの変更を検討した方が良いかもしれません。**

In [25]:
def filter_q1(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^問題[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^Problem[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"\n答え[：:].*", "", content, flags=re.DOTALL)
    content = re.sub(r"\n[解回]答[：:].*", "", content, flags=re.DOTALL)
    content = re.sub(r"\n[Aa]nswer[：:].*", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

実行して動作を確認します。

In [26]:
filter_q1(q1s[0])

'中学校の英語教師であるあなたは、創造的な執筆と文学の授業を行っています。最近、学生たちに人気のある映画をテーマにした短編小説を書く課題を出しました。クラスには30人の生徒がいて、各生徒は平均して5ページの短編小説を書く予定です。\n\nもしその短編小説をすべて印刷するための用紙が必要だとしたら、1ページあたりの用紙の価格が0.05ドルである場合、すべての生徒の短編小説を印刷するのに必要な総コストはいくらになるでしょうか？'

In [27]:
filter_q1("問題: ここは問題です。\n\n解答: ここは答えです。")

'ここは問題です。'

問題なさそうです。

作成した関数を組み合わせて、バッチ処理で `Q1` を生成する関数を作成します。

In [28]:
def generate_q1(
    llm: LanguageModel, tasks: list[str], topics: list[str], items: list[str], targets: list[str]
) -> list[str]:
    return [
        filter_q1(q1)
        for q1 in llm([
            get_q1_prompt(task, topic, item, target)
            for task, topic, item, target in zip(tasks, topics, items, targets)
        ])
    ]

実行して動作を確認します。

In [29]:
q1s = generate_q1(
    llm,
    ["math", "advanced math"],
    ["persona", "persona"],
    [random.choice(personas), random.choice(personas)],
    ["grade school student", "graduate student"],
)
pp.pprint(q1s)

['ある湿地帯には、2種類の非在来グラスが生えています。Aグラスは湿地の面積の60%を占めており、Bグラスは残りの40%を占めています。この湿地の面積は500平方メートルです。AグラスとBグラスのそれぞれが占める面積を求めなさい。',
 '一様な電場 \\( \\mathbf{E} \\) が \\( z \\) 軸に沿って存在し、強度は \\( E_0 \\) です。この電場の中にある帯電粒子が、初期速度 \\( \\mathbf{v_0} = v_{0x} '
 '\\hat{\\mathbf{i}} + v_{0y} \\hat{\\mathbf{j}} \\) で \\( xy \\) 平面内を運動しています。粒子の質量を \\( m \\)、電荷を \\( q \\) とします。\n'
 '\n'
 '1. 粒子が電場の影響を受けて運動する場合、運動方程式を立て、粒子の加速度 \\( \\mathbf{a} \\) を求めなさい。\n'
 '2. 粒子の運動を \\( t \\) 時間後の位置 \\( \\mathbf{r}(t) \\) と速度 \\( \\mathbf{v}(t) \\) を用いて表現し、時間 \\( t \\) における \\( x '
 '\\)、\\( y \\)、\\( z \\) 座標の数式を示しなさい。\n'
 '\n'
 'ここで、\\( \\hat{\\mathbf{i}} \\), \\( \\hat{\\mathbf{j}} \\), \\( \\hat{\\mathbf{k}} \\) はそれぞれ \\( x \\), \\( y \\), '
 '\\( z \\) 軸方向の単位ベクトルです。']


問題なさそうです。ここまでで `Q1` を生成する関数の実装が完了しました。

## A1 の生成

次に、`A1` の生成を行います。ここでは基本的に `Q1` を入力して、長さ指定(`簡潔に`)と、出力言語の指定(`日本語で`)をして言語モデルに生成させます。言語モデルによっては、表現を微調整する必要があるかもしれません。

実行の流れは `Q1` と同様ですので、ここではコードのみ記載します。

In [30]:
def get_a1_prompt(q1: str) -> list[dict]:
    A1_PROMPT_TEMPLATE = "{q1}\n\n簡潔に日本語で回答してください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": A1_PROMPT_TEMPLATE.format(q1=q1)},
    ]

In [31]:
a1_prompt = get_a1_prompt(q1s[0])
pp.pprint(a1_prompt)

[{'content': 'ある湿地帯には、2種類の非在来グラスが生えています。Aグラスは湿地の面積の60%を占めており、Bグラスは残りの40%を占めています。この湿地の面積は500平方メートルです。AグラスとBグラスのそれぞれが占める面積を求めなさい。\n'
             '\n'
             '簡潔に日本語で回答してください。',
  'role': 'user'}]


In [32]:
a1s = llm([get_a1_prompt(q1s[0])])
pp.pprint(a1s)

['Aグラスが占める面積は、500平方メートルの60%である300平方メートルです。Bグラスが占める面積は、残りの40%である200平方メートルです。']


In [33]:
a1s = llm([get_a1_prompt(q1s[1])])
pp.pprint(a1s)

['1. 粒子が電場 \\( \\mathbf{E} \\) の影響を受ける場合、運動方程式は次のようになります。\n'
 '\n'
 '\\[\n'
 'm \\frac{d\\mathbf{v}}{dt} = q \\mathbf{E}\n'
 '\\]\n'
 '\n'
 'ここで、電場 \\( \\mathbf{E} = E_0 \\hat{\\mathbf{k}} \\) なので、右辺は \\( q E_0 \\hat{\\mathbf{k}} \\) となります。このことから、加速度 \\( '
 '\\mathbf{a} \\) は以下のように求められます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{a} = \\frac{d\\mathbf{v}}{dt} = \\frac{q E_0}{m} \\hat{\\mathbf{k}}\n'
 '\\]\n'
 '\n'
 'つまり、粒子の加速度は \\( z \\) 軸方向に \\( \\frac{q E_0}{m} \\) です。\n'
 '\n'
 '2. 初期速度を考慮すると、粒子の速度 \\( \\mathbf{v}(t) \\) は次のように表現できます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{v}(t) = v_{0x} \\hat{\\mathbf{i}} + v_{0y} \\hat{\\mathbf{j}} + \\left( \\frac{q E_0}{m} t \\right) '
 '\\hat{\\mathbf{k}}\n'
 '\\]\n'
 '\n'
 '位置 \\( \\mathbf{r}(t) \\) は速度の積分によって求められます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{r}(t) = \\left( v_{0x} t \\right) \\hat{\\mathbf{i}} + \\left( v_{0y} t \\right) \\hat{\\mathbf{j}} + '
 '\\left( \\frac{1}{2} \\cdot \\frac{q E_0}{m} t^2 \\right) \\hat{\\mathbf{k}} + \\mathbf{r_0}\n'
 '\\]\n'
 '\n'
 'ここで、\

In [34]:
def filter_a1(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^答え[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[解回]答[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[Aa]nswer[：:]", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

In [35]:
filter_a1("解答: ここは答えです。")

'ここは答えです。'

In [36]:
def generate_a1(llm: LanguageModel, q1s: list[str]) -> list[str]:
    return [filter_a1(a1) for a1 in llm([get_a1_prompt(q1) for q1 in q1s])]

In [37]:
a1s = generate_a1(llm, q1s)
pp.pprint(a1s)

['Aグラスが占める面積は、500平方メートルの60%である300平方メートルです。Bグラスが占める面積は、500平方メートルの40%である200平方メートルです。',
 '1. 粒子が電場 \\( \\mathbf{E} \\) の影響を受ける場合、運動方程式は次のようになります。\n'
 '\n'
 '\\[\n'
 'm \\frac{d\\mathbf{v}}{dt} = q \\mathbf{E}\n'
 '\\]\n'
 '\n'
 'ここで、電場 \\( \\mathbf{E} = E_0 \\hat{\\mathbf{k}} \\) なので、右辺は \\( q E_0 \\hat{\\mathbf{k}} \\) となります。このことから、加速度 \\( '
 '\\mathbf{a} \\) は以下のように求められます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{a} = \\frac{d\\mathbf{v}}{dt} = \\frac{q E_0}{m} \\hat{\\mathbf{k}}\n'
 '\\]\n'
 '\n'
 'つまり、粒子の加速度は \\( z \\) 軸方向に \\( \\frac{q E_0}{m} \\) です。\n'
 '\n'
 '2. 初期速度を考慮すると、粒子の速度 \\( \\mathbf{v}(t) \\) は次のように表現できます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{v}(t) = v_{0x} \\hat{\\mathbf{i}} + v_{0y} \\hat{\\mathbf{j}} + \\left( \\frac{q E_0}{m} t \\right) '
 '\\hat{\\mathbf{k}}\n'
 '\\]\n'
 '\n'
 '位置 \\( \\mathbf{r}(t) \\) は速度の積分によって求められます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{r}(t) = \\left( v_{0x} t \\right) \\hat{\\mathbf{i}} + \\left( v_{0y} t \\right) \\hat{\\mathbf{j}} + '
 '\\left( \\frac{1}{2} \\cdot \\fra

## Q2 の生成

次に、追加質問 `Q2` の生成を行います。

- まず `Q1` に関連する質問であることを強調します(`前述の問題をより理解するために`の部分)。これが無いと `Q2` で新たな別の質問を生成してしまうことがあります。
- 次に `問題の一部を変更したり、条件を追加しても良いです` という部分で、`Q1` の問題の一部を変更することを促します。これにより、`Q2` が `Q1` に関連する質問となる確率を高めることができます。
- 最後に答えを含まないように注意を促します(`決して答えを含めないでください`の部分)。

In [38]:
def get_q2_prompt(q1: str, a1: str) -> list[dict[str, str]]:
    Q2_PROMPT_TEMPLATE = "前述の問題をより理解するために、簡潔な追加の質問を一つ作ってください。問題の一部を変更したり、条件を追加しても良いです。追加の質問だけを書き、決して答えを含めないでください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": q1},
        {"role": "assistant", "content": a1},
        {"role": "user", "content": Q2_PROMPT_TEMPLATE},
    ]

In [39]:
q2_prompt = get_q2_prompt(q1s[0], a1s[0])
pp.pprint(q2_prompt)

[{'content': 'ある湿地帯には、2種類の非在来グラスが生えています。Aグラスは湿地の面積の60%を占めており、Bグラスは残りの40%を占めています。この湿地の面積は500平方メートルです。AグラスとBグラスのそれぞれが占める面積を求めなさい。',
  'role': 'user'},
 {'content': 'Aグラスが占める面積は、500平方メートルの60%である300平方メートルです。Bグラスが占める面積は、500平方メートルの40%である200平方メートルです。', 'role': 'assistant'},
 {'content': '前述の問題をより理解するために、簡潔な追加の質問を一つ作ってください。問題の一部を変更したり、条件を追加しても良いです。追加の質問だけを書き、決して答えを含めないでください。',
  'role': 'user'}]


In [40]:
q2s = llm([get_q2_prompt(q1s[0], a1s[0])])
pp.pprint(q2s)

['湿地帯の面積が700平方メートルに増えた場合、AグラスとBグラスのそれぞれが占める面積は何平方メートルになりますか？']


In [41]:
def filter_q2(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^追加の質問[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^質問[：:]", "", content, flags=re.DOTALL)
    content = content.strip()
    return content

In [42]:
def generate_q2(llm: LanguageModel, q1s: list[str], a1s: list[str]) -> list[str]:
    return [filter_q2(q2) for q2 in llm([get_q2_prompt(q1, a1) for q1, a1 in zip(q1s, a1s)])]

In [43]:
q2s = generate_q2(llm, q1s, a1s)
pp.pprint(q2s)

['湿地帯の面積が800平方メートルに増えた場合、AグラスとBグラスのそれぞれが占める面積は何平方メートルになりますか？',
 '帯電粒子が電場 \\( \\mathbf{E} \\) の中で運動しているとき、粒子が初期位置 \\( (x_0, y_0, z_0) \\) から \\( t \\) 時間後に到達する位置 \\( (x(t), y(t), '
 'z(t)) \\) が、電場の強度 \\( E_0 \\) が時間とともに変化し、 \\( E(t) = E_0(1 + kt) \\) （\\( k \\) は定数）となる場合、粒子の加速度 \\( \\mathbf{a}(t) '
 '\\) を求めなさい。']


一旦、ここまでの対話データの流れを確認します。

In [44]:
pp.pprint(list(zip(q1s, a1s, q2s))[0])

('ある湿地帯には、2種類の非在来グラスが生えています。Aグラスは湿地の面積の60%を占めており、Bグラスは残りの40%を占めています。この湿地の面積は500平方メートルです。AグラスとBグラスのそれぞれが占める面積を求めなさい。',
 'Aグラスが占める面積は、500平方メートルの60%である300平方メートルです。Bグラスが占める面積は、500平方メートルの40%である200平方メートルです。',
 '湿地帯の面積が800平方メートルに増えた場合、AグラスとBグラスのそれぞれが占める面積は何平方メートルになりますか？')


In [45]:
pp.pprint(list(zip(q1s, a1s, q2s))[1])

('一様な電場 \\( \\mathbf{E} \\) が \\( z \\) 軸に沿って存在し、強度は \\( E_0 \\) です。この電場の中にある帯電粒子が、初期速度 \\( \\mathbf{v_0} = v_{0x} '
 '\\hat{\\mathbf{i}} + v_{0y} \\hat{\\mathbf{j}} \\) で \\( xy \\) 平面内を運動しています。粒子の質量を \\( m \\)、電荷を \\( q \\) とします。\n'
 '\n'
 '1. 粒子が電場の影響を受けて運動する場合、運動方程式を立て、粒子の加速度 \\( \\mathbf{a} \\) を求めなさい。\n'
 '2. 粒子の運動を \\( t \\) 時間後の位置 \\( \\mathbf{r}(t) \\) と速度 \\( \\mathbf{v}(t) \\) を用いて表現し、時間 \\( t \\) における \\( x '
 '\\)、\\( y \\)、\\( z \\) 座標の数式を示しなさい。\n'
 '\n'
 'ここで、\\( \\hat{\\mathbf{i}} \\), \\( \\hat{\\mathbf{j}} \\), \\( \\hat{\\mathbf{k}} \\) はそれぞれ \\( x \\), \\( y \\), '
 '\\( z \\) 軸方向の単位ベクトルです。',
 '1. 粒子が電場 \\( \\mathbf{E} \\) の影響を受ける場合、運動方程式は次のようになります。\n'
 '\n'
 '\\[\n'
 'm \\frac{d\\mathbf{v}}{dt} = q \\mathbf{E}\n'
 '\\]\n'
 '\n'
 'ここで、電場 \\( \\mathbf{E} = E_0 \\hat{\\mathbf{k}} \\) なので、右辺は \\( q E_0 \\hat{\\mathbf{k}} \\) となります。このことから、加速度 \\( '
 '\\mathbf{a} \\) は以下のように求められます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{a} = \\frac{d\\mathbf{v}}{dt} = \\frac{q E_0}{m} \\hat{\\math

問題なさそうです。

## A2 の生成

最後に、`A2` の生成を行います。基本的に `A1` の生成と同様ですので、ここではコードのみ記載します。

In [46]:
def get_a2_prompt(q1: str, a1: str, q2: str) -> list[dict[str, str]]:
    A2_PROMPT_TEMPLATE = "{q2}\n\n簡潔に日本語で回答してください。"
    return [
        # {"role": "system", "content": "あなたは親切なAIアシスタントです。日本語で回答してください。"},
        {"role": "user", "content": q1},
        {"role": "assistant", "content": a1},
        {"role": "user", "content": A2_PROMPT_TEMPLATE.format(q2=q2)},
    ]

In [47]:
a2_prompt = get_a2_prompt(q1s[0], a1s[0], q2s[0])
pp.pprint(a2_prompt)

[{'content': 'ある湿地帯には、2種類の非在来グラスが生えています。Aグラスは湿地の面積の60%を占めており、Bグラスは残りの40%を占めています。この湿地の面積は500平方メートルです。AグラスとBグラスのそれぞれが占める面積を求めなさい。',
  'role': 'user'},
 {'content': 'Aグラスが占める面積は、500平方メートルの60%である300平方メートルです。Bグラスが占める面積は、500平方メートルの40%である200平方メートルです。', 'role': 'assistant'},
 {'content': '湿地帯の面積が800平方メートルに増えた場合、AグラスとBグラスのそれぞれが占める面積は何平方メートルになりますか？\n\n簡潔に日本語で回答してください。', 'role': 'user'}]


In [48]:
a2s = llm([get_a2_prompt(q1, a1, q2) for q1, a1, q2 in zip(q1s, a1s, q2s)])
pp.pprint(a2s)

['Aグラスは480平方メートル（800平方メートルの60%）、Bグラスは320平方メートル（800平方メートルの40%）を占めます。',
 '電場 \\( \\mathbf{E}(t) \\) が時間とともに変化し、\\( E(t) = E_0(1 + kt) \\) と表される場合、粒子にかかる力は次のようになります。\n'
 '\n'
 '\\[\n'
 '\\mathbf{F}(t) = q \\mathbf{E}(t) = q E_0(1 + kt) \\hat{\\mathbf{k}}\n'
 '\\]\n'
 '\n'
 '運動方程式 \\( m \\frac{d\\mathbf{v}}{dt} = \\mathbf{F}(t) \\) に基づき、加速度 \\( \\mathbf{a}(t) \\) は次のように求められます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{a}(t) = \\frac{\\mathbf{F}(t)}{m} = \\frac{q E_0(1 + kt)}{m} \\hat{\\mathbf{k}}\n'
 '\\]\n'
 '\n'
 'したがって、粒子の加速度は以下のようになります。\n'
 '\n'
 '\\[\n'
 '\\mathbf{a}(t) = \\frac{q E_0}{m}(1 + kt) \\hat{\\mathbf{k}}\n'
 '\\]']


In [49]:
def filter_a2(content: str) -> str:
    content = content.strip()
    content = re.sub(r"^答え[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[解回]答[：:]", "", content, flags=re.DOTALL)
    content = re.sub(r"^[Aa]nswer[：:]", "", content, flags=re.DOTALL)  # cspell: disable-line
    content = content.strip()
    return content

In [50]:
def generate_a2(llm: LanguageModel, q1s: list[str], a1s: list[str], q2s: list[str]) -> list[str]:
    return [filter_a2(a2) for a2 in llm([get_a2_prompt(q1, a1, q2) for q1, a1, q2 in zip(q1s, a1s, q2s)])]

In [51]:
a2s = generate_a2(llm, q1s, a1s, q2s)
pp.pprint(a2s)

['Aグラスは480平方メートル（800平方メートルの60%）、Bグラスは320平方メートル（800平方メートルの40%）を占めます。',
 '電場 \\( \\mathbf{E}(t) \\) が時間とともに変化し、\\( E(t) = E_0(1 + kt) \\) と表される場合、粒子にかかる力は次のようになります。\n'
 '\n'
 '\\[\n'
 '\\mathbf{F}(t) = q \\mathbf{E}(t) = q E_0(1 + kt) \\hat{\\mathbf{k}}\n'
 '\\]\n'
 '\n'
 '運動方程式 \\( m \\frac{d\\mathbf{v}}{dt} = \\mathbf{F}(t) \\) に基づき、加速度 \\( \\mathbf{a}(t) \\) は次のように求められます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{a}(t) = \\frac{\\mathbf{F}(t)}{m} = \\frac{q E_0(1 + kt)}{m} \\hat{\\mathbf{k}}\n'
 '\\]\n'
 '\n'
 'したがって、粒子の加速度は以下のようになります。\n'
 '\n'
 '\\[\n'
 '\\mathbf{a}(t) = \\frac{q E_0}{m}(1 + kt) \\hat{\\mathbf{k}}\n'
 '\\]']


再度ここまでの対話データの流れを確認します。

In [52]:
pp.pprint(list(zip(q1s, a1s, q2s, a2s))[0])

('ある湿地帯には、2種類の非在来グラスが生えています。Aグラスは湿地の面積の60%を占めており、Bグラスは残りの40%を占めています。この湿地の面積は500平方メートルです。AグラスとBグラスのそれぞれが占める面積を求めなさい。',
 'Aグラスが占める面積は、500平方メートルの60%である300平方メートルです。Bグラスが占める面積は、500平方メートルの40%である200平方メートルです。',
 '湿地帯の面積が800平方メートルに増えた場合、AグラスとBグラスのそれぞれが占める面積は何平方メートルになりますか？',
 'Aグラスは480平方メートル（800平方メートルの60%）、Bグラスは320平方メートル（800平方メートルの40%）を占めます。')


In [53]:
pp.pprint(list(zip(q1s, a1s, q2s, a2s))[1])

('一様な電場 \\( \\mathbf{E} \\) が \\( z \\) 軸に沿って存在し、強度は \\( E_0 \\) です。この電場の中にある帯電粒子が、初期速度 \\( \\mathbf{v_0} = v_{0x} '
 '\\hat{\\mathbf{i}} + v_{0y} \\hat{\\mathbf{j}} \\) で \\( xy \\) 平面内を運動しています。粒子の質量を \\( m \\)、電荷を \\( q \\) とします。\n'
 '\n'
 '1. 粒子が電場の影響を受けて運動する場合、運動方程式を立て、粒子の加速度 \\( \\mathbf{a} \\) を求めなさい。\n'
 '2. 粒子の運動を \\( t \\) 時間後の位置 \\( \\mathbf{r}(t) \\) と速度 \\( \\mathbf{v}(t) \\) を用いて表現し、時間 \\( t \\) における \\( x '
 '\\)、\\( y \\)、\\( z \\) 座標の数式を示しなさい。\n'
 '\n'
 'ここで、\\( \\hat{\\mathbf{i}} \\), \\( \\hat{\\mathbf{j}} \\), \\( \\hat{\\mathbf{k}} \\) はそれぞれ \\( x \\), \\( y \\), '
 '\\( z \\) 軸方向の単位ベクトルです。',
 '1. 粒子が電場 \\( \\mathbf{E} \\) の影響を受ける場合、運動方程式は次のようになります。\n'
 '\n'
 '\\[\n'
 'm \\frac{d\\mathbf{v}}{dt} = q \\mathbf{E}\n'
 '\\]\n'
 '\n'
 'ここで、電場 \\( \\mathbf{E} = E_0 \\hat{\\mathbf{k}} \\) なので、右辺は \\( q E_0 \\hat{\\mathbf{k}} \\) となります。このことから、加速度 \\( '
 '\\mathbf{a} \\) は以下のように求められます。\n'
 '\n'
 '\\[\n'
 '\\mathbf{a} = \\frac{d\\mathbf{v}}{dt} = \\frac{q E_0}{m} \\hat{\\math

面白い問題かどうかはともかく、問題なさそうです。これで `A2` の生成も完了しました。

## マルチターンの対話データの生成

これまで作成した関数を組み合わせて、マルチターンの対話データを生成する関数を作成します。結果は OpenAI messages 形式で出力します。

In [54]:
def synthesis_multi_turn_qa(
    llm: LanguageModel, tasks: list[str], topics: list[str], items: list[str], targets: list[str]
) -> list[dict[str, str | list[dict[str, str]]]]:
    q1s = generate_q1(llm, tasks, topics, items, targets)
    a1s = generate_a1(llm, q1s)
    q2s = generate_q2(llm, q1s, a1s)
    a2s = generate_a2(llm, q1s, a1s, q2s)
    return [
        {
            "messages": [
                {"role": "user", "content": q1},
                {"role": "assistant", "content": a1},
                {"role": "user", "content": q2},
                {"role": "assistant", "content": a2},
            ],
            "task": task,
            "topic": topic,
            "item": item,
            "target": target,
        }
        for task, topic, item, target, q1, a1, q2, a2 in zip(tasks, topics, items, targets, q1s, a1s, q2s, a2s)
    ]

実行して動作を確認します。

In [55]:
qas = synthesis_multi_turn_qa(
    llm,
    ["math"],
    ["persona"],
    [random.choice(personas)],
    ["grade school student"],
)
pp.pprint(qas)

[{'item': 'A science historian or a chemistry teacher focused on the history of scientific discoveries.',
  'messages': [{'content': 'ある化学の教師は、科学の発見に関する歴史を教えています。彼は、17世紀の科学者が発見した元素の数を5つ、18世紀の科学者が発見した元素の数を7つ、19世紀の科学者が発見した元素の数を10個学んでいます。彼が教えた全ての元素の合計は何個になりますか？',
                'role': 'user'},
               {'content': '17世紀の元素5つ、18世紀の元素7つ、19世紀の元素10個を合計すると、5 + 7 + 10 = 22個になります。したがって、全ての元素の合計は22個です。',
                'role': 'assistant'},
               {'content': '20世紀の科学者が発見した元素の数を5つ追加した場合、全ての元素の合計は何個になりますか？', 'role': 'user'},
               {'content': '合計は22 + 5 = 27個になります。', 'role': 'assistant'}],
  'target': 'grade school student',
  'task': 'math',
  'topic': 'persona'}]


この関数をバッチ処理で実行し、結果を JSONL ファイルに保存する関数を作成します。`task`, `topic`, `item`, `target` は、与えられた引数からランダムに選択して、対話データを生成します。

In [56]:
def run_synthesis_multi_turn_qa(
    llm: LanguageModel,
    task_list: list[str],
    topic_list: list[str],
    item_list: list[str],
    target_list: list[str],
    num_samples: int = 1,
    batch_size: int = 1,
    output_jsonl: str = "output.jsonl",
):
    with open(output_jsonl, "a", encoding="utf-8") as f:
        for i in range(num_samples // batch_size):
            tasks = [random.choice(task_list) for _ in range(batch_size)]
            topics = [random.choice(topic_list) for _ in range(batch_size)]
            items = [random.choice(item_list) for _ in range(batch_size)]
            targets = [random.choice(target_list) for _ in range(batch_size)]
            assert len(tasks) == len(topics) == len(items) == len(targets) == batch_size
            qas = synthesis_multi_turn_qa(llm, tasks, topics, items, targets)
            for qa in qas:
                f.write(json.dumps(qa, ensure_ascii=False) + "\n")
                logger.debug(pp.pformat(qa))

ペルソナデータから対話データを生成します。

In [57]:
run_synthesis_multi_turn_qa(
    llm,
    ["math", "arithmetic", "basic math", "basic arithmetic"],
    ["persona"],
    personas,
    ["grade school student"],
    1,
    1,
    "output.jsonl",
)

{'item': 'An elementary school teacher or a parent educator focused on developing good study habits in young '
         'children.',
 'messages': [{'content': 'ある小学校の先生が、子どもたちに毎日15分間の勉強時間を設けることにしました。もし、5日間勉強した場合、合計で何分間勉強したことになりますか？', 'role': 'user'},
              {'content': '5日間で合計75分間勉強したことになります。', 'role': 'assistant'},
              {'content': 'もし、毎日の勉強時間を20分に増やした場合、5日間で合計何分間勉強したことになりますか？', 'role': 'user'},
              {'content': '5日間で合計100分間勉強したことになります。', 'role': 'assistant'}],
 'target': 'grade school student',
 'task': 'basic math',
 'topic': 'persona'}


難易度を上げて生成します。

In [58]:
run_synthesis_multi_turn_qa(
    llm,
    ["advanced math"],
    ["persona"],
    personas,
    ["graduate student"],
    1,
    1,
    "output.jsonl",
)

{'item': 'A public health professional with a focus on infectious diseases and epidemiology or a tropical medicine '
         'specialist, likely working in a region with high Dengue fever prevalence.',
 'messages': [{'content': 'ある地域で、デング熱の発生率が年間で1,200件と報告されています。公衆衛生専門家であるあなたは、この疾病の伝播を減少させるために、地域住民の50%にワクチン接種を行うことを計画しています。ワクチン接種後、感染率が接種した住民の間で30%減少すると仮定します。\n'
                          '\n'
                          '1. ワクチン接種前の感染率は、接種を受けた住民と受けていない住民を考慮した場合、全体の年間感染率に対してどのように変化しますか？\n'
                          '2. ワクチン接種後、年間の感染件数は何件になると予想されますか？\n'
                          '\n'
                          '接種を受ける住民の割合、接種後の感染率の減少を考慮し、計算を行ってください。',
               'role': 'user'},
              {'content': '1. '
                          'ワクチン接種前の感染率は、年間1,200件の感染が報告されています。接種を受ける住民は50%で、接種を受けない住民も50%です。このため、接種を受けた住民の感染率は全体の年間感染率に影響を与えます。具体的には、接種を受けた住民の感染件数と接種を受けない住民の感染件数を合計し、全体の感染率に反映されますが、現時点では具体的な数値は示されていません。\n'
                          '\n'
                          '2. '
              

## まとめ

TODO

## 参考文献

TODO

以上